In [1]:
from bbSearchRaw import SearchProblem, search

Loading bbSearch Version 2.1 (at 21:54, Wed 26 Mar)
Last module source code edit 9am Thursday 24th Feb 2022


In [2]:
class Robot:
    def __init__(self, location, carried_items, strength):
        self.location      = location
        self.carried_items = carried_items
        self.strength      = strength

    def weight_carried(self):
        return sum([ITEM_WEIGHT[i] for i in self.carried_items])

    ## Define unique string representation for the state of the robot object
    def __repr__(self):
        return str( ( self.location,
                      self.carried_items,
                      self.strength ) )

class Door:
    def __init__(self, roomA, roomB, doorkey=None, locked=False):
        self.goes_between = {roomA, roomB}
        self.doorkey      = doorkey
        self.locked       = locked
        # Define handy dictionary to get room on other side of a door
        self.other_loc = {roomA:roomB, roomB:roomA}

    ## Define a unique string representation for a door object
    def __repr__(self):
        return str( ("door", self.goes_between, self.doorkey, self.locked) )

In [3]:
class State:
    def __init__( self, robot, doors, room_contents ):
        self.robot = robot
        self.doors = doors
        self.room_contents = room_contents

    ## Define a string representation that will be uniquely identify the state.
    ## An easy way is to form a tuple of representations of the components of
    ## the state, then form a string from that:
    def __repr__(self):
        return str( ( self.robot.__repr__(),
                      [d.__repr__() for d in self.doors],
                      self.room_contents ) )


In [5]:
from copy import deepcopy

class RobotWorker( SearchProblem ):

    def __init__( self, state, goal_item_locations ):
        self.initial_state = state
        self.goal_item_locations = goal_item_locations

    def possible_actions( self, state ):

        robot_location = state.robot.location
        strength       = state.robot.strength
        weight_carried = state.robot.weight_carried()

        actions = []
        # Can put down any carried item
        for i in state.robot.carried_items:
            actions.append( ("put down", i) )

        # Can pick up any item in room if strong enough修改
        for i in state.room_contents[robot_location]:
            if ITEM_WEIGHT[i]==0:
                actions.append( ("pick up", i))
            elif any(i in goal_items for goal_items in self.goal_item_locations.values()):
                if strength >= weight_carried + ITEM_WEIGHT[i]:
                    actions.append(("pick up", i))

        # If there is an unlocked door between robot location and
        # another location can move to that location
#         print(state.doors)
        for door in state.doors:
            if  door.locked==False and robot_location in door.goes_between:
                actions.append( ("move to", door.other_loc[robot_location]) )
            elif door.locked==True and door.doorkey in state.robot.carried_items and robot_location in door.goes_between:
#                 print(door)
#                 print(robot_location)
#                 print(door.doorkey)
#                 print(door.other_loc[robot_location])
                actions.append( ("move to", door.other_loc[robot_location]) )
        # Now the actions list should contain all possible actions
        return actions

    def successor( self, state, action):
        next_state = deepcopy(state)
        act, target = action
        if act== "put down":
            next_state.robot.carried_items.remove(target)
            next_state.room_contents[state.robot.location].add(target)

        if act == "pick up":
            next_state.robot.carried_items.append(target)
            next_state.room_contents[state.robot.location].remove(target)

        if act == "move to":
            next_state.robot.location = target

        return next_state

    def goal_test(self, state):
        #print(state.room_contents)
        for room, contents in self.goal_item_locations.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    return False
        return True

    def display_state(self,state):
        print("Robot location:", state.robot.location)
        print("Robot carrying:", state.robot.carried_items)
        print("Room contents:", state.room_contents)

## middle

In [4]:
# ROOM_CONTENTS = {
#     'workshop'     : {'rusty key'},
#     'store room'   : {'bucket', 'suitcase'},
#     'tool cupboard' : {'sledge hammer', 'anvil', 'saw', 'screwdriver'},
# }

# ITEM_WEIGHT = {
#       'rusty key' : 0,
#          'bucket' : 2,
#        'suitcase' : 4,
#     'screwdriver' : 1,
#   'sledge hammer' : 5,
#           'anvil' : 12,
#             'saw' : 2,
# }

# DOORS = [
#     Door('workshop', 'store room' ),
#     Door( 'store room', 'tool cupboard', doorkey='rusty key', locked=False )
# ]
ROOM_CONTENTS = {
    'workshop'     : {'thing3'},
    'store room'   : {'bucket', 'suitcase','key2','rusty key'},
    'tool cupboard' : {'sledge hammer', 'anvil', 'saw', 'screwdriver'},
    'room1': {'thing1', 'thing2'}
}

ITEM_WEIGHT = {
      'rusty key' : 0,
         'bucket' : 2,
       'suitcase' : 4,
    'screwdriver' : 1,
  'sledge hammer' : 5,
          'anvil' : 12,
            'saw' : 2,
    'thing1':3,
    'thing2':8,
    'thing3':10,
    'key2':0
}

DOORS = [
    Door('workshop', 'store room' ),
    Door( 'store room', 'tool cupboard', doorkey='rusty key', locked=True),
    Door( 'workshop', 'room1', doorkey='key2', locked=True ),
    Door('room1','tool cupboard')
]


In [6]:
# rob = Robot('store room', [], 15 )

# state = State(rob, DOORS, ROOM_CONTENTS)

# goal_item_locations =  {"store room":{"sledge hammer", "screwdriver", "anvil"}}

# RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )
rob = Robot('store room', [], 15 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {
    "workshop":{"thing1"},
    "store room":{"sledge hammer", "screwdriver", "anvil"},
    "room1" :{"bucket"}
}

RW_PROBLEM_1 = RobotWorker( state, goal_item_locations )

## hard

In [15]:
ROOM_CONTENTS = {
    'workshop'     : {'rusty key'},
    'store room'   : {'bucket', 'suitcase','key2'},
    'tool cupboard' : {'sledge hammer', 'anvil', 'saw', 'screwdriver'},
    'room1': {'thing1', 'thing2','thing3'}
}

ITEM_WEIGHT = {
      'rusty key' : 0,
         'bucket' : 2,
       'suitcase' : 4,
    'screwdriver' : 1,
  'sledge hammer' : 5,
          'anvil' : 12,
            'saw' : 2,
    'thing1':3,
    'thing2':8,
    'thing3':10,
    'key2':0
}

DOORS = [
    Door('workshop', 'store room' ),
    Door( 'store room', 'tool cupboard', doorkey='rusty key', locked=True),
    Door( 'workshop', 'room1', doorkey='key2', locked=True ),
    Door('room1','tool cupboard')
]


In [16]:
rob = Robot('store room', [], 15 )

state = State(rob, DOORS, ROOM_CONTENTS)

goal_item_locations =  {
    "workshop":{"thing1"},
    "store room":{"sledge hammer", "screwdriver", "anvil"},
    "room1" :{"bucket"}
}

RW_PROBLEM_2 = RobotWorker( state, goal_item_locations )

## 查看可能的动作

In [7]:
poss_acts = RW_PROBLEM_1.possible_actions( RW_PROBLEM_1.initial_state )
poss_acts

[('pick up', 'key2'),
 ('pick up', 'rusty key'),
 ('pick up', 'bucket'),
 ('move to', 'workshop')]

In [8]:
for act in poss_acts:
    print("Action", act, "leads to the following state:")
    next_state = RW_PROBLEM_1.successor( RW_PROBLEM_1.initial_state, act )
    RW_PROBLEM_1.display_state(next_state)
    print()

Action ('pick up', 'key2') leads to the following state:
Robot location: store room
Robot carrying: ['key2']
Room contents: {'workshop': {'thing3'}, 'store room': {'rusty key', 'bucket', 'suitcase'}, 'tool cupboard': {'sledge hammer', 'anvil', 'screwdriver', 'saw'}, 'room1': {'thing1', 'thing2'}}

Action ('pick up', 'rusty key') leads to the following state:
Robot location: store room
Robot carrying: ['rusty key']
Room contents: {'workshop': {'thing3'}, 'store room': {'key2', 'bucket', 'suitcase'}, 'tool cupboard': {'sledge hammer', 'anvil', 'screwdriver', 'saw'}, 'room1': {'thing1', 'thing2'}}

Action ('pick up', 'bucket') leads to the following state:
Robot location: store room
Robot carrying: ['bucket']
Room contents: {'workshop': {'thing3'}, 'store room': {'key2', 'rusty key', 'suitcase'}, 'tool cupboard': {'sledge hammer', 'anvil', 'screwdriver', 'saw'}, 'room1': {'thing1', 'thing2'}}

Action ('move to', 'workshop') leads to the following state:
Robot location: workshop
Robot carr

## 没有启发式

In [18]:
search( RW_PROBLEM_1, 'BF/FIFO', 100000, loop_check=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=None
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
.........................................................
!! Search node limit (100000) reached !!
): No solution found :(


SEARCH SPACE STATS:
Total nodes generated          =   306595  (includes start)
Nodes discarded by loop_check  =   206594  (100001 distinct states added to queue)
Nodes tested (by goal_test)    =    57710  (all expanded)
Nodes left in queue            =    42290

Time taken = 43.4444 seconds



'NODE_LIMIT_EXCEEDED'

## 启发式

In [11]:
def heuristic(state,goal_position):
    total_distance=0
    for room, contents in state.room_contents.items():
        if room in goal_position:
            for i in contents:
                if i not in goal_position[room]:
                    total_distance +=1 
    return total_distance

In [12]:
search( RW_PROBLEM_1, 'BF/FIFO', 100000, loop_check=True, heuristic=heuristic,show_state_path=True)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=heuristic
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)
..........................
:-)) *SUCCESS* ((-:

Path length = 19
Goal state is:
Robot location: workshop
Robot carrying: ['key2', 'rusty key']
Room contents: {'workshop': {'thing3', 'thing1'}, 'store room': {'sledge hammer', 'screwdriver', 'anvil', 'suitcase'}, 'tool cupboard': {'saw'}, 'room1': {'bucket', 'thing2'}}
The action path to the solution is:
    ('pick up', 'key2')
    ('pick up', 'rusty key')
    ('pick up', 'bucket')
    ('move to', 'workshop')
    ('move to', 'room1')
    ('pick up', 'thing1')
    ('put down', 'bucket')
    ('move to', 'tool cupboard')
    ('pick up', 'sledge hammer')
    ('pick up', 'screwdriver')
    ('move to', 'store room')
    ('put down', 'sl

'GOAL_STATE_FOUND'

In [13]:
def heuristic2 (state,goal_position):
    total_distance=0
    for room, contents in goal_position.items():
            for i in contents:
                if not i in state.room_contents[room]:
                    total_distance +=1 
    return total_distance

In [14]:
search( RW_PROBLEM_1, 'BF/FIFO', 100000, loop_check=True, heuristic=heuristic2)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=heuristic2
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'rusty key']
Room contents: {'workshop': {'thing3', 'thing1'}, 'store room': {'sledge hammer', 'screwdriver', 'anvil', 'suitcase'}, 'tool cupboard': {'saw'}, 'room1': {'bucket', 'thing2'}}
The action path to the solution is:
    ('pick up', 'key2')
    ('pick up', 'bucket')
    ('move to', 'workshop')
    ('move to', 'room1')
    ('put down', 'bucket')
    ('pick up', 'thing1')
    ('move to', 'workshop')
    ('put down', 'thing1')
    ('move to', 'store room')
    ('pick up', 'rusty key')
    ('move to', 'tool cupboard')
    ('pick up', 'sledge hammer')
    ('move to', 'st

'GOAL_STATE_FOUND'

In [17]:
search( RW_PROBLEM_2, 'BF/FIFO', 100000, loop_check=True, heuristic=heuristic2)

This is the general SearchProblem parent class
You must extend this class to encode a particular search problem.

** Running Brandon's Search Algorithm **
Strategy: mode=BF/FIFO, cost=None, heuristic=heuristic2
Max search nodes: 100000  (max number added to queue)
Searching (will output '.' each 1000 goal_tests)

:-)) *SUCCESS* ((-:

Path length = 22
Goal state is:
Robot location: store room
Robot carrying: ['key2', 'rusty key']
Room contents: {'workshop': {'thing1'}, 'store room': {'sledge hammer', 'screwdriver', 'anvil', 'suitcase'}, 'tool cupboard': {'saw'}, 'room1': {'thing3', 'bucket', 'thing2'}}
The action path to the solution is:
    ('pick up', 'key2')
    ('pick up', 'bucket')
    ('move to', 'workshop')
    ('move to', 'room1')
    ('put down', 'bucket')
    ('pick up', 'thing1')
    ('move to', 'workshop')
    ('put down', 'thing1')
    ('pick up', 'rusty key')
    ('move to', 'store room')
    ('move to', 'tool cupboard')
    ('pick up', 'sledge hammer')
    ('move to', 'st

'GOAL_STATE_FOUND'